In [4]:
import requests
import numpy as np
import pandas as pd 
import geopandas as gpd

In [11]:
def get_geosphere_data(parameters: list = "T2M",
                       start: str = '2021-08-01T12:00', 
                       end: str = '2021-08-02T12:00', 
                       lat_lon: list = ['48.206248, 16.367569'], 
                       output_format: str = "geojson"):
    """
    function for retrieving data from geosphere API
    """
    
    base_url = 'https://dataset.api.hub.zamg.ac.at/v1/timeseries/historical/inca-v1-1h-1km'
    lat_lon_params = '&lat_lon='.join(lat_lon)
    url = f'{base_url}?parameters={parameters}&start={start}&end={end}&lat_lon={lat_lon_params}&output_format={output_format}'
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Request failed with status code {response.status_code}.')

In [12]:
data = get_geosphere_data()

https://dataset.api.hub.zamg.ac.at/v1/timeseries/historical/inca-v1-1h-1km?parameters=T2M&start=2021-08-01T12:00&end=2021-08-02T12:00&lat_lon=48.206248, 16.367569&output_format=geojson


In [3]:
# function
# input bounding box and get data (temp, prec, rhum, wind) for last 24 hours from day of interest (DOI) noon - day before DOI noon 

def get_geosphere_data(parameters: str = "T2M",
                       day_of_interest: str = '2021-08-01T12:00',
                       bbox: list = [int], 
                       output_format: str = "geojson"):
    

    url = "https://dataset.api.hub.geosphere.at/v1/grid/historical/inca-v1-1h-1km?parameters=RR&parameters=T2M&start=2021-01-01T00%3A00&end=2021-01-01T00%3A00&bbox=47.45%2C14.05%2C47.50%2C14.10&output_format=geojson" 
    base_url = "https://dataset.api.hub.geosphere.at/v1/grid/historical/inca-v1-1h-1km"
    url = f"{base_url}?parameters=RR&parameters=T2M"
"
    lat_lon_params = '&lat_lon='.join(lat_lon)
    url = f'{base_url}?parameters={parameters}&start={start}&end={end}&lat_lon={lat_lon_params}&output_format={output_format}'
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Request failed with status code {response.status_code}.')

In [5]:
path_to_test_aoi = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\fwi_data\test_aoi.shp"
aoi = gpd.read_file(path_to_test_aoi)

In [10]:
data = get_geosphere_data("T2M", '2021-08-01T12:00', aoi.bounds.iloc[0], "geojson")